### Rosetta SilentStruct r.m.s.d Calculations

In [ ]:
# import modules 
import sys
sys.path.append('path/to/dl_binder_design')  # library from :https://github.com/bcov77/silent_tools 
import numpy as np
import MDAnalysis as mda
from include.silent_tools import silent_tools 
from pyrosetta.rosetta.std import stringstream
import numpy as np
from pyrosetta.rosetta.core.io.silent import SilentFileData, SilentFileOptions
from pyrosetta.rosetta.std import stringstream
from pyrosetta import *
from rosetta import *
init( '-in:file:silent_struct_type binary -mute all' )

In [11]:
# Function to calculate RMSD after translation and rotation using the Kabsch algorithm
def kabsch_rmsd(P, Q):
    """Calculate the RMSD after optimal rotational alignment (Kabsch algorithm)."""
    
    # Calculate the centroids of P and Q
    centroid_P = np.mean(P, axis=0)
    centroid_Q = np.mean(Q, axis=0)
    
    # Center the coordinates by subtracting the centroid
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q
    
    # Perform singular value decomposition to get the optimal rotation matrix
    C = np.dot(P_centered.T, Q_centered)
    V, S, W = np.linalg.svd(C)
    
    # Compute the optimal rotation matrix
    d = (np.linalg.det(V) * np.linalg.det(W)) < 0.0
    if d:
        S[-1] = -S[-1]
        V[:, -1] = -V[:, -1]
    
    U = np.dot(V, W)
    
    # Rotate the centered coordinates of P
    P_rotated = np.dot(P_centered, U)
    
    # Calculate the RMSD between the rotated P and Q
    diff = P_rotated - Q_centered
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    
    return rmsd


In [ ]:
# Getting data for original pdb
original_pdb = 'native.pdb'
E2_motif = mda.Universe(original_pdb)
E2_backbone = E2_motif.select_atoms('resid 1-4 or resid 5-10 and name CA + N + C + O and chainID A')
E2_Ca = E2_motif.select_atoms('resid 1-4 or resid 5-10 and name CA and chainID A')


In [ ]:
# input

silent_filepath = "path/to/file/xyz.silent"
Motif_res = ['position of fixed residues'] # fixed residues. 

motif_residues = []
for resi in Motif_res:
    motif_residues.append(int(resi))

# Auto Process for calculations of Ca rmsd's 
silent_index = silent_tools.get_silent_index( silent_filepath)
struct_iterator = silent_tools.get_silent_index(silent_filepath)['tags']
sfd_in = rosetta.core.io.silent.SilentFileData(rosetta.core.io.silent.SilentFileOptions())
sfd_in.read_file(silent_filepath)
res = sfd_in.get_structure(struct_iterator[0]) # We only have one structure but different structure can be mentioned.
design_pose = Pose()
res.fill_pose(design_pose)

# Initialize the design_backbone array
design_backbone = np.zeros((len(motif_residues) * 4, 3), dtype=np.float64) # storing CA+N+C+O coordinates
Motif_CA_coords = np.zeros((len(motif_residues) , 3) , dtype=np.float64) # Storing CA coordinates 

# Loop through the list of residues
for index, i in enumerate(motif_residues):
    residue = design_pose.residue(i)
    ca_coords = residue.xyz('CA')
    N_coords = residue.xyz('N')
    C_coords = residue.xyz('C')
    O_coords = residue.xyz('O')

    # Motif CA coordinates
    Motif_CA_coords[index] = ca_coords
    # Store coordinates in the design_backbone array
    design_backbone[index * 4] = N_coords
    design_backbone[index * 4 + 1] = ca_coords
    design_backbone[index * 4 + 2] = C_coords
    design_backbone[index * 4 + 3] = O_coords

ca_rmsd = kabsch_rmsd(E2_Ca.positions , Motif_CA_coords)
bb_rmsd = kabsch_rmsd(E2_backbone.positions, design_backbone)
print(f" Backbone RMSD: {bb_rmsd:.3f} Å")
print(f" CA RMSD: {ca_rmsd:.3f} Å")